** We are going to be working through understanding and interacting with the PCP Plugin framework.  The SamplePlugin is copied below from the prior SamplePlugin notebook. **

** The two core functions will be to: **
* Request jobs
* Respond with job output

** The start function should be an infinite loop handling these two processes. **
** The functionality will need to be a list of valid commands. If you have built the commands for your plugin, from the Introduction.ipynb, copy and paste these commands into this notebook. **
** WARNING:  The framework may terminate your thread, but it will call your _stop() function when the container is stopped. **

** The sample command is pasted below and the command variable (cmdWget) is pasted into your SamplePlugin Class to demonstrate building the command(s) into your custom plugin. **

In [ ]:
try:
    from src.controller_plugin import ControllerPlugin
except ImportError:
    from sys import path
    path.append("/root/backend-interpreter/plugin_interpreter")
    print(path)
    from src.controller_plugin import ControllerPlugin
    from os import environ
    environ["PORT"] = "10000" 

In [ ]:
cmdWget = {"CommandName" : "wget",
          "Tooltip" : "Retrieves raw webpage files from targeted systems",
          "Output" : True,
          "Inputs" : [{"Name" : "Target IP Address",
                     "Type" : "url",
                     "Tooltip" : "Modify the URL to the desired target IP address",
                     "Value" : "http://192.168.1.100"}]}

In [ ]:
class SamplePlugin(ControllerPlugin):
    functionality =  [cmdWget]                         # Get the commands you built from the prior tutorial
    def __init__(self):
        """
        The plugin class is the last class to init,
        but Database and logger have not started yet
        Should not be using the database handles or the logger in this fn
        """
        self.name = "SamplePlugin"
        super().__init__(self.name, self.functionality)
    def _start(self):
        """
        Start function is called by the base class and is the container entrypoint
        This function will assume the db is already started
        This function should include all calls required to run until _stop is called.

        :return: This function calls exit! when complete
        """
        #################################################################################################
        # Add Plugin specific startup routine here
        
        pass
        ################################################################################################
        # Add plugin control loop here
        while True:
            # Assume you have a connected client for this controller
            new_job = self.request_job()  # <dict> or None  
            if not None:  # use your plugin functionality to get this job to your connected client
                output_content = None # or <add code for the plugin content for this target/job (string obj)
                # Give this content back to the framework
                self.respond_output(job, output_content)
            sleep(3)                                        
        
    def _stop(self):
        ###############################################################################################
        # Put cleanup code here
        ###############################################################################################
        
        pass

In [ ]:
SamplePlugin()